In [61]:
import numpy as np
from numpy import linalg as la

# Naivni algoritam
Ovaj algoritam mozemo primeniti na 4 para tacaka (original-slika) sto ga cini manje preciznim u odnosu na obican i normalizovani DLT algoritam

In [62]:
#P = [0  3  5
#     4  0  0
#    -1 -1  6]

#ova funkcija nam sluzi da izracunamo koeficijente alfa, beta i gama iz sistema D = alfa*A + beta*B + gama*C
def koeficijenti(matrica):
    a = matrica[0]
    b = matrica[1]
    c = matrica[2]
    d = matrica[3]

    #zapisujemo sistem u obliku A*X = B

    A = [[a[0], b[0], c[0]],
         [a[1], b[1], c[1]],
         [a[2], b[2], c[2]]
        ]

    B = [d[0],
         d[1],
         d[2]
        ]

    #X dobijamo kao proizvod A_inv i B
    A_inv = la.inv(A)

    X = np.matmul(A_inv, B)

    return X
    

In [63]:
def naivni_algoritam(original, slika):
    #racunamo koeficijente alfa, beta i gama za originale
    p1 = koeficijenti(original)

    #racunamo koeficijente alfa, beta i gama za slike
    p2 = koeficijenti(slika)

    #matrica p1 ce kao kolone imati alfa * A, beta * B i gama * C
    p1 = [[p1[0] * x for x in original[0]],
          [p1[1] * x for x in original[1]],
          [p1[2] * x for x in original[2]]
        ]
    p1 = np.transpose(p1)

    #matrica p2 ce kao kolone imati alfa' * A', beta' * B' i gama' * C'
    p2 = [[p2[0] * x for x in slika[0]],
          [p2[1] * x for x in slika[1]],
          [p2[2] * x for x in slika[2]]
        ]

    p2 = np.transpose(p2)

    #trazena matrica preslikavanja
    p = np.matmul(p2,la.inv(p1))

    return p

In [64]:
originali = [[-3,2,1],
             [-2,5,2], 
             [1,0,3], 
             [-7,3,1]
            ]

slike = [[11,-12,7],
         [25,-8,9],
         [15,4,17], 
         [14,-28,10]
        ]
#Pokrecemo algoritam za test primere
P1 = naivni_algoritam(originali, slike)
P1 = np.round(P1, decimals=5)
print("Matrica projektivnog preslikavanja dobijena naivnim algoritmom:\n")
print(P1)


Matrica projektivnog preslikavanja dobijena naivnim algoritmom:

[[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]]


Nakon pokretanja Naivnog algoritma vidimo da smo dobili istu matricu kao i polaznu

# DLT ALGORITAM

In [65]:
#Funkcija koja primenjuje preslikavanje dato matricom matrica na niz tacaka tacke
def primeni(tacke, matrica):
    n = len(tacke)
    nove_t = []
    for i in range(n):
         nove_t.append(np.matmul(matrica, tacke[i]))
    return nove_t

In [66]:
#Funkcija koja racuna matricu za jednu korespodenciju
def matrica_korespodencije(o, s):
    m = np.matrix([[0, 0, 0, -s[2]*o[0], -s[2]*o[1], -s[2]*o[2], s[1]*o[0], s[1]*o[1], s[1]*o[2]],
     [s[2]*o[0], s[2]*o[1], s[2]*o[2], 0, 0, 0, -s[0]*o[0], -s[0]*o[1], -s[0]*o[2]]])

    return m

In [74]:
def dlt_algoritam(n, originali, slike):
    matrica = []

    #Za sve korespodencije originala i slike odredjujemo 2x9 matricu a potom ih spajamo u jednu 2n*9
    for i in range(n):
        if i > 0:
            m = matrica_korespodencije(originali[i], slike[i])
            matrica = np.concatenate((matrica, m), axis=0)
        else:
            matrica = matrica_korespodencije(originali[i], slike[i])
    
    #Radimo SVD dekompoziciju matrice 
    U, D, Vt = la.svd(matrica, full_matrices=True)
    
    #Matrica P ce biti poslednja kolona matrice V sto je ustvari poslednja vrsta matrica Vt
    P = Vt[-1]
    P = P.reshape(3,3)

    return P

In [77]:
originali = [[-3,2,1],
             [-2,5,2], 
             [1,0,3], 
             [-7,3,1],
             [2,1,2],
             [-1,2,1],
             [1,1,1]
            ]

slike = [[11,-12,7],
         [25,-8,9],
         [15,4,17], 
         [14,-28,10],
         [13,8,9],
         [11,-4,5],
         [8.02,4,4]
        ]

#Pokrecemo DLT algoritam za 6 tacaka    
matrica = dlt_algoritam(7, originali, slike)
matrica = np.round(matrica,decimals=10)

print("Matrica projektivnog preslikavanja dobijena dlt algoritmom")
print(matrica, "\n")

#Da bismo poredili sa matricom dobijenom naivnim algoritmom, skaliramo. m_naivni[0][1] je 3
matrica = matrica/matrica[0][1]*3
print("Skalirana matrica projektivnog preslikavanja dobijena dlt algoritmom")
print(matrica, "\n")
#Kada zaokruzimo na 2 decimale poklapaju se, jer smo sliku izmenili na drugoj decimali
matrica_round = np.round(matrica, decimals=2)

print("Matrica projektivnog preslikavanja dobijena dlt algoritmom skalirana i zaokruzena na 2 decimale")
print(matrica_round, "\n")

matrica_promene = [[0,1,2], [-1,0,3], [0,0,1]]
originali_prom = primeni(originali, matrica_promene)
slike_prom = primeni(slike, matrica_promene)


matrica1 = dlt_algoritam(7, originali_prom, slike_prom)
matrica1 = np.round(matrica1,decimals=10)

matrica_vrati = la.inv(matrica_promene)
matrica1 = np.matmul(matrica_vrati, matrica1)
matrica1 = np.matmul(matrica1, matrica_promene)
print("Matrica preslikavanja u starom koordinatnom sistemu:\n")
print(matrica1, "\n")

matrica1 = matrica1/matrica1[0][1]*3
print("Reskalirana matrica preslikavanja:\n")
print(matrica1, "\n")

print("Matrica izracunata pre promene koordinata:\n")
print(matrica, "\n")

Matrica projektivnog preslikavanja dobijena dlt algoritmom
[[ 5.30529000e-05  3.20011319e-01  5.32929220e-01]
 [ 4.26445525e-01 -2.02187000e-05 -2.95145000e-05]
 [-1.06595323e-01 -1.06519060e-01  6.39542445e-01]] 

Skalirana matrica projektivnog preslikavanja dobijena dlt algoritmom
[[ 4.97353345e-04  3.00000000e+00  4.99603472e+00]
 [ 3.99778538e+00 -1.89543608e-04 -2.76688651e-04]
 [-9.99295809e-01 -9.98580867e-01  5.99549835e+00]] 

Matrica projektivnog preslikavanja dobijena dlt algoritmom skalirana i zaokruzena na 2 decimale
[[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]] 

Matrica preslikavanja u starom koordinatnom sistemu:

[[-1.73564000e-05 -1.04847554e-01 -1.74607666e-01]
 [-1.39719503e-01  6.61760000e-06  9.63360000e-06]
 [ 3.49246285e-02  3.48997143e-02 -2.09538156e-01]] 

Reskalirana matrica preslikavanja:

[[ 4.96618165e-04  3.00000000e+00  4.99604405e+00]
 [ 3.99779004e+00 -1.89349195e-04 -2.75645915e-04]
 [-9.99297373e-01 -9.98584504e-01  5.99550913e+00]] 

Matrica izracu

Vidimo da se resenje dobijeno DLT algoritmom i ono koje smo dobili Naivnim algoritmom poklapaju na dve decimale, mi smo sliku poslednje tacke pokvarili na drugoj decimali tako da to ima smisla.

Prilikom testiranja invarijantnosti u odnosu na promenu koordinata, primetimo da se matrica pre i ona nakon promene koordinata malo razlikuju, pa zakljucujemo da DLT algoritam nije invarijantan u odnosu na promenu koordinata

# Normalizovani DLT algoritam

In [78]:
def afina(tacka):
    return [tacka[0]/tacka[2], tacka[1]/tacka[2]]

In [79]:
def homogena(tacka):
    return [tacka[0], tacka[1], 1]

In [80]:
#Funkcija koja vraca matricu normalizacije tacaka i normalizovane tacke
def normalizacija(n,tacke):
    afine_tacke = []

    #Odredjujemo teziste tacaka
    cx = 0
    cy = 0
    
    for i in range(n):
        afine_tacke.append(afina(tacke[i]))
        cx = cx + afine_tacke[i][0]
        cy = cy + afine_tacke[i][1]
    cx = cx/n
    cy = cy/n

    #Pravimo matricu koja translira teziste u koordinatni pocetak
    G = [[1,0,-cx], [0,1,-cy], [0,0,1]]

    #Svaku tacku transliramo za vektor CO
    for i in range(n):
        afine_tacke[i] = np.matmul(G, homogena(afine_tacke[i]))
   
    #Racunamo prosecno rastojanje tacaka od koordinatnog pocetka
    dist = 0
    for i in range(n):
        dist = dist + np.sqrt(np.square(afine_tacke[i][0]) + np.square(afine_tacke[i][1]))

    dist = dist/n

    #Pravimo matricu homotetije kako bismo skalirali tacke da prosecna udaljenost bude koren iz dva
    S = [[np.sqrt(2)/dist,0, 0], [0,np.sqrt(2)/dist,0], [0,0,1]]

    #Skaliramo tacke
    for i in range(n):
        afine_tacke[i] = np.matmul(S, homogena(afine_tacke[i]))

    #Matricu normalizacije dobijamo kao proizvod matrice homotetije i matrice translacije
    T = np.matmul(S,G)

    return T

In [81]:
#Normalizovani DLT algoritam
def dltnorm_algoritam(n,originali, slike):

    #Vrsimo normalizaciju originalnih tacaka
    originali_t = normalizacija(n,originali)

    #Vrsimo normalizaciju slika tacaka
    slike_t = normalizacija(n,slike)

    originali_n = primeni(originali, originali_t)
  
    slike_n = primeni(slike, slike_t)

    #print("Matrica za normalizaciju originala")
    #print(originali_t)

    #print("Matrica za normalizaciju slika")
    #print(slike_t)

    #Dobijamo matricu P' tako sto primenjujemo obican DLT algoritam na normalizovane tacke
    matrica_p = dlt_algoritam(n, originali_n, slike_n)

    #Matricu projektivnog preslikavanja P dobijamo kao proizvod inverza matrice normalizacije slika, P' i matrice normalizacije originala
    matrica = np.matmul(matrica_p, originali_t)
    matrica = np.matmul(la.inv(slike_t), matrica)

    return np.round(matrica,decimals=10)

In [82]:

originali = [[-3,2,1],
             [-2,5,2], 
             [1,0,3], 
             [-7,3,1],
             [2,1,2],
             [-1,2,1],
             [1,1,1]
            ]

slike = [[11,-12,7],
         [25,-8,9],
         [15,4,17], 
         [14,-28,10],
         [13,8,9],
         [11,-4,5],
         [8.02,4,4]
        ]
        
matrica = dltnorm_algoritam(7,originali, slike)
print("Matrica projektivnog preslikavanja dobijena normalizovanim dlt algoritmom\n")
print(matrica, "\n")

print("Skalirana matrica projektivnog preslikavanja dobijena normalizovanim dlt algoritmom\n")
matrica = matrica/matrica[0][1]*3
print(matrica, "\n")

matrica_round = np.round(matrica, decimals=2)
print(matrica_round, "\n")

matrica_promene = [[0,1,2], [-1,0,3], [0,0,1]]

originali_prom = primeni(originali, matrica_promene)
slike_prom = primeni(slike, matrica_promene)

matrica1 = dltnorm_algoritam(7, originali_prom, slike_prom)
matrica1 = np.round(matrica1,decimals=10)

matrica_vrati = la.inv(matrica_promene)
matrica1 = np.matmul(matrica_vrati, matrica1)
matrica1 = np.matmul(matrica1, matrica_promene)
print("Matrica preslikavanja u starom koordinatnom sistemu\n")
print(matrica1, "\n")

matrica1 = matrica1/matrica1[0][1]*3
print("Reskalirana matrica preslikavanja\n")
print(matrica1, "\n")

print("Matrica izracunata pre promene koordinata\n")
print(matrica, "\n")


Matrica projektivnog preslikavanja dobijena normalizovanim dlt algoritmom

[[ 3.89785000e-05  2.34697334e-01  3.90851377e-01]
 [ 3.12756186e-01 -1.48614000e-05 -2.16908000e-05]
 [-7.81772458e-02 -7.81212009e-02  4.69041798e-01]] 

Skalirana matrica projektivnog preslikavanja dobijena normalizovanim dlt algoritmom

[[ 4.98239575e-04  3.00000000e+00  4.99602662e+00]
 [ 3.99778107e+00 -1.89964663e-04 -2.77260925e-04]
 [-9.99294424e-01 -9.98578035e-01  5.99548946e+00]] 

[[ 0.  3.  5.]
 [ 4. -0. -0.]
 [-1. -1.  6.]] 

Matrica preslikavanja u starom koordinatnom sistemu

[[-3.89784000e-05 -2.34697334e-01 -3.90851377e-01]
 [-3.12756187e-01  1.48615000e-05  2.16913000e-05]
 [ 7.81772458e-02  7.81212009e-02 -4.69041798e-01]] 

Reskalirana matrica preslikavanja

[[ 4.98238296e-04  3.00000000e+00  4.99602663e+00]
 [ 3.99778107e+00 -1.89965941e-04 -2.77267316e-04]
 [-9.99294424e-01 -9.98578034e-01  5.99548946e+00]] 

Matrica izracunata pre promene koordinata

[[ 4.98239575e-04  3.00000000e+00  4.

Vidimo da se matrica dobijena normalizovanim DLT algoritmom poklapa sa onom koju smo dobili pomocu DLT algoritma, kao i da se poklapa sa matricom dobijenom Naivnim algoritmom na dve decimale 

Sto se tice invarijantnosti u odnosu na promenu koordinata, vidimo da se matrice ne razlikuju, tj. da normalizovan tj modifikovan dlt algoritam jeste invarijantan u odnosu na promenu koordinata
